In [ ]:
from warnings import filters
import pandas as pd
import polars as pl
import duckdb
from datetime import datetime
import glob

In [54]:
# parameters
SINGLE_FILE = 'data/taxi/yellow_tripdata_2022-01.parquet'
MULTI_FILE_PATH = "data/taxi/yellow*.parquet"
MULTI_FILES = glob.glob(MULTI_FILE_PATH)

COLUMNS = ["tpep_pickup_datetime"]
READ_SINGLE_FILE = False

if READ_SINGLE_FILE:
    FILES = SINGLE_FILE
else:
    FILES = MULTI_FILE_PATH

# init
con = duckdb.connect()

# Functions

In [58]:
#polars
def polars_read_all():
    return pl.scan_parquet(FILES)
def polars_filter_all():
    return (
        pl.scan_parquet(FILES)
        .filter(pl.col("tpep_pickup_datetime").dt.month() >= 7)
    )
def polars_filter_one():
    return (
        pl.scan_parquet(FILES)
        .filter(pl.col("tpep_pickup_datetime").dt.month() >= 7)
        .select(["tpep_pickup_datetime"])
        
    )
def polars_filter_count():
    return (
        pl.scan_parquet(FILES)
        .filter(pl.col("tpep_pickup_datetime").dt.month() >= 7)
        .select(pl.len())
    )

polars_queries = {
    "polars_read_multi": polars_read_all(),
    "polars_filter_all": polars_filter_all(),
    "polars_filter_one": polars_filter_one(),
    "polars_filter_count": polars_filter_count()
}

#duckdb
duckdb_queries = {
    "duckdb_read_all": f"""
        SELECT *
        FROM '{FILES}'
    """,
    "duckdb_filter_all": f"""
        SELECT *
        FROM '{FILES}'
        WHERE EXTRACT(MONTH FROM tpep_pickup_datetime) >= 7
    """,
    "duckdb_filter_one": f"""
        SELECT tpep_pickup_datetime
        FROM '{FILES}'
        WHERE EXTRACT(MONTH FROM tpep_pickup_datetime) >= 7
    """,
    "duckdb_filter_count": f"""
        SELECT count(*)
        FROM '{FILES}'
        WHERE EXTRACT(MONTH FROM tpep_pickup_datetime) >= 7
    """
}


# Query Visualisation

In [59]:
#polars
def explain_polars_query(lazyframe, label):
    print(f"\n--- {label} ---")
    print(lazyframe.explain(optimized=True))  # Set to False for raw plan

for name, lf in polars_queries.items():
    explain_polars_query(lf, name)


--- polars_read_multi ---
Parquet SCAN [data\taxi\yellow_tripdata_2022-01.parquet, ... 11 other sources] [id: 2268918374784]
PROJECT */19 COLUMNS

--- polars_filter_all ---
Parquet SCAN [data\taxi\yellow_tripdata_2022-01.parquet, ... 11 other sources] [id: 2268918375936]
PROJECT */19 COLUMNS
SELECTION: [(col("tpep_pickup_datetime").dt.month()) >= (7)]

--- polars_filter_one ---
Parquet SCAN [data\taxi\yellow_tripdata_2022-01.parquet, ... 11 other sources] [id: 2268918375248]
PROJECT 1/19 COLUMNS
SELECTION: [(col("tpep_pickup_datetime").dt.month()) >= (7)]

--- polars_filter_count ---
SELECT [len()]
  Parquet SCAN [data\taxi\yellow_tripdata_2022-01.parquet, ... 11 other sources] [id: 2268918373312]
  PROJECT 2/19 COLUMNS
  SELECTION: [(col("tpep_pickup_datetime").dt.month()) >= (7)]


In [57]:
#duckdb
def explain_duckdb_query(query: str, label: str):
    print(f"\n--- {label} ---")
    res = con.sql(query)
    print(res.explain())  # Logical plan

for name, query in duckdb_queries.items():
    explain_duckdb_query(query, name)


--- duckdb_read_all ---




--- duckdb_filter_all ---




--- duckdb_filter_one ---




--- duckdb_filter_count ---
